In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"]="true"

os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")

In [2]:
from langchain_groq import ChatGroq
llm = ChatGroq(
     model_name="llama3-8b-8192" 
)

Data Ingestion--From the website we need to scrap the data


In [3]:
from langchain_community.document_loaders import WebBaseLoader


USER_AGENT environment variable not set, consider setting it to identify your requests.


In [4]:
url="https://python.langchain.com/docs/tutorials/"

In [5]:
loader=WebBaseLoader(url)

In [6]:
text=loader.load()
text

[Document(metadata={'source': 'https://python.langchain.com/docs/tutorials/', 'title': 'Tutorials | ğŸ¦œï¸�ğŸ”— LangChain', 'description': 'New to LangChain or LLM app development in general? Read this material to quickly get up and running building your first applications.', 'language': 'en'}, page_content='\n\n\n\n\nTutorials | ğŸ¦œï¸�ğŸ”— LangChain\n\n\n\n\n\n\nSkip to main contentJoin us at  Interrupt: The Agent AI Conference by LangChain on May 13 & 14 in San Francisco!IntegrationsAPI ReferenceMoreContributingPeopleError referenceLangSmithLangGraphLangChain HubLangChain JS/TSv0.3v0.3v0.2v0.1ğŸ’¬SearchIntroductionTutorialsBuild a Question Answering application over a Graph DatabaseTutorialsBuild a simple LLM application with chat models and prompt templatesBuild a ChatbotBuild a Retrieval Augmented Generation (RAG) App: Part 2Build an Extraction ChainBuild an AgentTaggingBuild a Retrieval Augmented Generation (RAG) App: Part 1Build a semantic search engineBuild a Question/Answering

In [7]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
split=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)


In [8]:
docs=split.split_documents(text)

In [9]:
docs

[Document(metadata={'source': 'https://python.langchain.com/docs/tutorials/', 'title': 'Tutorials | ğŸ¦œï¸�ğŸ”— LangChain', 'description': 'New to LangChain or LLM app development in general? Read this material to quickly get up and running building your first applications.', 'language': 'en'}, page_content='Tutorials | ğŸ¦œï¸�ğŸ”— LangChain'),
 Document(metadata={'source': 'https://python.langchain.com/docs/tutorials/', 'title': 'Tutorials | ğŸ¦œï¸�ğŸ”— LangChain', 'description': 'New to LangChain or LLM app development in general? Read this material to quickly get up and running building your first applications.', 'language': 'en'}, page_content='Skip to main contentJoin us at  Interrupt: The Agent AI Conference by LangChain on May 13 & 14 in San Francisco!IntegrationsAPI ReferenceMoreContributingPeopleError referenceLangSmithLangGraphLangChain HubLangChain JS/TSv0.3v0.3v0.2v0.1ğŸ’¬SearchIntroductionTutorialsBuild a Question Answering application over a Graph DatabaseTutorialsBuild a

In [28]:
from langchain_community.embeddings import HuggingFaceEmbeddings
embedding=HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2')

In [11]:
from langchain_community.vectorstores import FAISS
db=FAISS.from_documents(documents=docs,embedding=embedding)

In [12]:
query="what LangChain components from a specific provider?"

In [13]:
result=db.similarity_search(query)
print(result[0].page_content)

New to LangChain or LLM app development in general? Read this material to quickly get up and running building your first applications.
Get startedâ€‹
Familiarize yourself with LangChain's open-source components by building simple applications.
If you're looking to get started with chat models, vector stores,
or other LangChain components from a specific provider, check out our supported integrations.


Retrieval Chain,Document chain

In [14]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

In [15]:
prompt=ChatPromptTemplate.from_template(
    """
    Answer the following questions based only the provided context:
    
    <context>
    {context}
    </context>
    
    """
)

In [16]:
document_chain=create_stuff_documents_chain(llm,prompt)

In [17]:
from langchain_core.documents import Document
document_chain.invoke({
                       
                      "input":"what LangChain components from a specific provider?",
                      "context":[Document(page_content="""Familiarize yourself with LangChain's open-source components by building simple applications.

If you're looking to get started with chat models, vector stores, or other LangChain components from a specific provider, check out our supported integrations.

Chat models and prompts: Build a simple LLM application with prompt templates and chat models.
Semantic search: Build a semantic search engine over a PDF with document loaders, embedding models, and vector stores.
Classification: Classify text into categories or labels using chat models with structured outputs.
Extraction: Extract structured data from text and other unstructured media using chat models and few-shot examples."""
      )]      })

"Based on the provided context, I can answer the following questions:\n\n1. What can you do to familiarize yourself with LangChain's open-source components?\nAnswer: You can build simple applications.\n\n2. Where can you find supported integrations for LangChain components?\nAnswer: You can find them with specific providers.\n\n3. What examples of applications can be built using LangChain's components?\nAnswer: You can build a simple LLM application with prompt templates and chat models, a semantic search engine over a PDF with document loaders, embedding models, and vector stores, classify text into categories or labels using chat models with structured outputs, extract structured data from text and other unstructured media using chat models and few-shot examples."

Retriver

In [18]:
db

In [19]:
retriver=db.as_retriever()

In [26]:
retriver

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001AF9EE21F40>, search_kwargs={})

In [20]:
from langchain.chains import create_retrieval_chain
retriver_chain=create_retrieval_chain(retriver,document_chain)

In [25]:
document_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\n    Answer the following questions based only the provided context:\n    \n    <context>\n    {context}\n    </context>\n    \n    '), additional_kwargs={})])
| ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001AF94EFC560>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001AF94EFD5B0>, model_name='llama3-8b-8192', model_kwargs={}, groq_api_key=SecretStr('**********'))
| StrOutputParser(), kwargs={}, config={'run_name': 'stuff_documents_chain'}, config_factories=[])

In [21]:
retriver_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001AF9EE21F40>, search_kwargs={}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\n    Answer the following questions based only the provided context:\n    \n    <context>\n    {context}\n    </context>\n    \n    '), additio

In [22]:
response=retriver_chain.invoke({
    "input":"what LangChain components from a specific provider?"
})

In [23]:
response['answer']

"I'm ready to answer your questions based on the provided context. Go ahead and ask away!"

In [24]:
response

{'input': 'what LangChain components from a specific provider?',
 'context': [Document(id='4bcf5a6d-301f-4405-81cd-479f57a42c19', metadata={'source': 'https://python.langchain.com/docs/tutorials/', 'title': 'Tutorials | ğŸ¦œï¸�ğŸ”— LangChain', 'description': 'New to LangChain or LLM app development in general? Read this material to quickly get up and running building your first applications.', 'language': 'en'}, page_content="New to LangChain or LLM app development in general? Read this material to quickly get up and running building your first applications.\nGet startedâ€‹\nFamiliarize yourself with LangChain's open-source components by building simple applications.\nIf you're looking to get started with chat models, vector stores,\nor other LangChain components from a specific provider, check out our supported integrations."),
  Document(id='d7cc7ab6-4471-4052-afbf-2fcdfdb85262', metadata={'source': 'https://python.langchain.com/docs/tutorials/', 'title': 'Tutorials | ğŸ¦œï¸�ğŸ”— Lan